Les codes dans ce colab sont des codes purement développés par moi Badreddine SAADIOUI et mon collègue Mohammed Akhermouch sous la supervision de notre tuteur Christian de Perreti dans le cadre d'un projet de recherche à l'École Centrale de Lyon. Grand merci pour monsieur Christian pour ces conseils et astuces précieux.

# **Stratégie achat-vente en utilisant Random Forest sur les directions**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Chargement des données depuis Yahoo Finance pour le ticker AMZN sur une base horaire
data = yf.download('ABB', start='2022-06-17', interval='1h', end='2024-06-10')

# Fonction pour calculer l'indicateur RSI (Relative Strength Index)
def calculate_RSI(data, window=14):
    delta = data['Adj Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    return 100 - (100 / (1 + RS))

# Ajout des indicateurs techniques
data['RSI'] = calculate_RSI(data)
data['EMA20'] = data['Adj Close'].ewm(span=20, adjust=False).mean()
data['EMA50'] = data['Adj Close'].ewm(span=50, adjust=False).mean()
data['EMA200'] = data['Adj Close'].ewm(span=200, adjust=False).mean()
data['Rendement'] = data['Adj Close'].pct_change() * 100

# Création des variables cibles pour la classification directionnelle
data['Adj Close +1'] = data['Adj Close'].shift(-1)
data['Direction'] = (data['Adj Close +1'] > data['Adj Close']).astype(int)
data.dropna(inplace=True)
print("Nouvelle base de données après transformation :")
print(data.head())

# Sélection des variables explicatives et de la variable cible
features = ['Open', 'High', 'Low', 'Adj Close', 'Volume', 'RSI', 'EMA20', 'EMA50', 'EMA200', 'Rendement']
X = data[features]
y = data['Direction']

# Séparation des données en ensembles d'entraînement et de validation
train_end_date = '2024-05-30 23:00:00'
validation_start_date = '2024-06-01 00:00:00'

train_data = data[data.index <= train_end_date]
validation_data = data[data.index >= validation_start_date]

X_train = train_data[features]
y_train = train_data['Direction']
X_valid = validation_data[features]
y_valid = validation_data['Direction']

# Division des données d'entraînement en ensembles d'entraînement et de test pour validation
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Initialisation du modèle de classification RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, min_samples_split=10, min_samples_leaf=5)
model.fit(X_train_split, y_train_split)

# Trading algorithmique du 2024-06-01 au 2024-06-10
initial_capital = 10000
capital = initial_capital
position = 0  # 0: pas de position, >0: détention d'actions
capital_history = []
profits = []
decisions = []
num_retrainings = 0

# Suivi des rendements horaires pour le calcul du Sharpe Ratio
hourly_returns = []

for i in range(len(validation_data)):
    current_data = validation_data.iloc[i:i+1]
    if current_data.empty:
        break

    X_current = current_data[features]
    y_current = current_data['Direction']

    # Prédiction de la direction pour l'heure suivante
    prediction = model.predict(X_current)[0]
    actual = y_current.values[0]
    decisions.append((current_data.index[0], prediction, actual))
    print(f'Time: {current_data.index[0]}, Prediction: {prediction}, Actual: {actual}, Retrainings: {num_retrainings}')

    # Prise de décision: Acheter, Vendre ou Rien faire
    if prediction == 1 and capital > 0:
        # Signal d'achat
        position = capital / current_data['Adj Close'].values[0]
        capital = 0
        profit = 0
    elif prediction == 0 and position > 0:
        # Signal de vente
        new_capital = position * current_data['Adj Close'].values[0]
        profit = new_capital - initial_capital
        capital = new_capital
        position = 0
    else:
        profit = 0

    profits.append(profit)
    hourly_returns.append(profit / initial_capital)  # Rendement horaire

    # Enregistrement du capital pour l'heure actuelle
    capital_history.append(capital + (position * current_data['Adj Close'].values[0]))

    # Réentraînement du modèle avec de nouvelles données
    X_train_split = pd.concat([X_train_split, X_current])
    y_train_split = pd.concat([y_train_split, y_current])
    model.fit(X_train_split, y_train_split)
    num_retrainings += 1

# Capital final si une position est détenue à la fin de la période
final_adj_close = validation_data['Adj Close'].iloc[-1]
if position > 0:
    capital = position * final_adj_close
capital_history.append(capital)

print(f'Capital final: ${capital:.2f}')
print(f'Return: {(capital - initial_capital) / initial_capital * 100:.2f}%')

# Calcul du Sharpe Ratio
hourly_returns = np.array(hourly_returns)
sharpe_ratio = np.mean(hourly_returns) / np.std(hourly_returns)
annualized_sharpe_ratio = sharpe_ratio * np.sqrt(252 * 24)  # 252 jours de trading par an, 24 heures par jour
print(f'Ratio de Sharpe annualisé: {annualized_sharpe_ratio:.2f}')

# Tracer l'évolution du capital
plt.figure(figsize=(14, 7))
plt.plot(capital_history)
plt.xlabel('Heure')
plt.ylabel('Capital ($)')
plt.title('Évolution du Capital au Fil du Temps')
plt.show()


[*********************100%%**********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ABB']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Nouvelle base de données après transformation :
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume, RSI, EMA20, EMA50, EMA200, Rendement, Adj Close +1, Direction]
Index: []


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

# **Bot de référence pour le CAC40**

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np

# Téléchargement des données depuis Yahoo Finance pour AMZN
ticker = 'ABB'
start_date = '2024-06-01'
end_date = '2024-06-10'
data = yf.download(ticker, start=start_date, end=end_date)

# Capital initial
initial_capital = 10000
capital = initial_capital

# Prix d'ouverture au début et à la fin de la période
initial_price = data['Open'].iloc[0]
final_price = data['Open'].iloc[-1]

# Nombre d'actions que l'on peut acheter avec le capital initial
num_shares = capital // initial_price

# Capital après la vente à la fin de la période
final_capital = num_shares * final_price

# Calcul du profit
profit = final_capital - initial_capital
profit_percentage = (profit / initial_capital) * 100  # Calcul du profit en pourcentage

print(f"Capital initial: ${initial_capital:.2f}")
print(f"Prix d'ouverture le {start_date}: ${initial_price:.2f}")
print(f"Nombre d'actions achetées: {num_shares}")
print(f"Prix d'ouverture le {end_date}: ${final_price:.2f}")
print(f"Capital final après vente: ${final_capital:.2f}")
print(f"Profit réalisé: ${profit:.2f} ({profit_percentage:.2f}%)")


# **Stratégie achat-vente en utilisant LSTM sur les directions**

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Charger les données de Yahoo Finance
data = yf.download('ABB', start='2022-06-17', interval='1h', end='2024-06-10')


# Ajouter des indicateurs
data['RSI'] = np.random.uniform(30, 70, len(data))
data['EMA20'] = data['Adj Close'].ewm(span=20, adjust=False).mean()
data['EMA50'] = data['Adj Close'].ewm(span=50, adjust=False).mean()
data['EMA200'] = data['Adj Close'].ewm(span=200, adjust=False).mean()
data['Rendement'] = data['Adj Close'].pct_change() * 100

# Créer la colonne 'Direction' : 1 si Adj Close +1 est plus élevé, 0 si inférieur ou égal
data['Adj Close +1'] = data['Adj Close'].shift(-1)
data['Direction'] = (data['Adj Close +1'] > data['Adj Close']).astype(int)

# Supprimer les lignes avec des valeurs NaN
data.dropna(inplace=True)

# Caractéristiques et cible
features = ['Open', 'High', 'Low', 'Adj Close', 'Volume', 'RSI', 'EMA20', 'EMA50', 'EMA200', 'Rendement']
X = data[features]
y = data['Direction']

# Diviser en ensembles d'entraînement et de validation
train_end_date = '2024-05-31'
validation_start_date = '2024-06-01'

train_data = data[data.index <= train_end_date]
validation_data = data[data.index >= validation_start_date]

X_train = train_data[features]
y_train = train_data['Direction']

# Normaliser les caractéristiques pour LSTM en utilisant la normalisation Z-score
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(validation_data[features])

# Reshape pour LSTM
X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_validation_scaled = X_validation_scaled.reshape((X_validation_scaled.shape[0], 1, X_validation_scaled.shape[1]))

# Construire le modèle LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())
# Entraîner le modèle LSTM
model.fit(X_train_scaled, y_train, epochs=10, batch_size=64, verbose=1)

capital_initiale = 10000
capital = capital_initiale
position = 0  # 0: aucune position, >0: détention d'actions
historique_capital = []

# Trading algorithmique du 2024-06-01 au 2024-06-10
for i in range(len(validation_data)):
    current_data = validation_data.iloc[i:i+1]
    if current_data.empty:
        break

    X_current = current_data[features]
    X_current_scaled = scaler.transform(X_current).reshape((1, 1, len(features)))

    # Prédire la direction pour l'heure suivante
    prediction = (model.predict(X_current_scaled) > 0.5).astype(int)[0][0]

    # Prise de décision : Acheter, Vendre ou Conserver
    if prediction == 1 and capital > 0:
        # Signal d'achat
        position = capital / current_data['Adj Close'].values[0]
        capital = 0
    elif prediction == 0 and position > 0:
        # Signal de vente
        capital = position * current_data['Adj Close'].values[0]
        position = 0

    # Enregistrer le capital pour l'heure actuelle
    historique_capital.append(capital + (position * current_data['Adj Close'].values[0]))

    # Réentraîner le modèle avec les nouvelles données
    X_train_scaled = np.append(X_train_scaled, X_current_scaled, axis=0)
    y_train = np.append(y_train, [current_data['Direction'].values[0]])
    model.fit(X_train_scaled, y_train, epochs=1, batch_size=64, verbose=0)

# Capital final si en position à la fin de la période
final_adj_close = validation_data['Adj Close'].iloc[-1]
if position > 0:
    capital = position * final_adj_close
historique_capital.append(capital)

print(f'Capital final : ${capital:.2f}')
print(f'Retour : {(capital - capital_initiale) / capital_initiale * 100:.2f}%')

# Plot de l'historique du capital
plt.plot(historique_capital)
plt.xlabel('Heure')
plt.ylabel('Capital ($)')
plt.title('Capital au fil du temps')
plt.show()

In [ ]:
pip install tensorflow pydot pydotplus graphviz


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
import pydotplus
from tensorflow.keras.utils import model_to_dot
from IPython.display import Image, display

# Define the model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(1, 1)),
    LSTM(50),
    Dense(1)
])

# Generate plot of the neural network architecture
tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',  # LR for horizontal, TB for vertical
    expand_nested=True,
    dpi=96  # Increase for higher resolution
)

# Display the model
display(Image('model.png'))


